<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/geemap_spectral_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jdT5ykPDlskIbgOR7-41Uph2HVYJGGHk1rjG6X3rg1E&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWizFS3PBlym3IVR3CP5-jWrsKRG1WDYiLXfqVBpgHswY6oYSXSnsF8

Successfully saved authorization token.


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
wdpa_polygon = ee.FeatureCollection('WCMC/WDPA/current/polygons')
# select the pa in cambodia
aoi = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Ang Trapeng Thmor'))


In [ ]:


#image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]')
image=ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi).filterDate('2020-01-01', '2020-12-31').sort('CLOUD_COVER').first().select('B[1-7]'))
landsat=image.clip(aoi)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

map4 = geemap.Map(center=[12.5657, 105],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(landsat, vis_params, 'Landsat-8')

map4.addLayerControl()
map4

Map(center=[12.5657, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [ ]:
## https://www.researchgate.net/publication/321685005_Spectral_normalization_between_Landsat-8OLI_Landsat-7ETM_and_CBERS-4MUX_bands_through_linear_regression_and_spectral_unmixing

In [ ]:
# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59] #values of urban vegetation in 7 landst bands
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]


In [ ]:
# Unmix the image.
#fractions = landsat.unmix([urban, veg, water])
fractions=landsat.unmix([urban,veg,water])

In [ ]:
Map = geemap.Map(center=[12,105], zoom= 8)
Map.addLayer(fractions.randomVisualizer())
Map

Map(center=[12, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [ ]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.12.1


In [ ]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
# Display the results.
tiles = {
    'image': image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128}),
    'unmixed': fractions.getMapId()
}

center = [12,105]
Mapdisplay(center, tiles, "Stamen Terrain")